In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


In [2]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Claire_set/"
if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)
    
all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders("/DATA/TransportROOT/DATA/1_year/", True)
clear_output(wait=False)
all_folders_drop


,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230329_Plate083_01/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,01/Img/
1,20230329_Plate083_02/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,02/Img/
2,20230329_Plate083_03/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,03/Img/
3,20230329_Plate057_02/Img/,057,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate057/0...,02/Img/
4,20230329_Plate057_03/Img/,057,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate057/0...,03/Img/
...,...,...,...,...,...
279,20230512_Plate118_008/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,008/Img/
280,20230512_Plate118_009/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,009/Img/
281,20230512_Plate118_010/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,010/Img/
282,20230512_Plate118_011/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,011/Img/


In [3]:
txt_drop[0][:-13]

'/DATA/MYRISTATE/DATA/1_year/20230512_Plate047/001/'

In [22]:
txt_addresses = []
for file in txt_drop:
    file_name = f"{file.split('/')[-3]}{os.sep}{file.split('/')[-2]}{os.sep}{file.split('/')[-1]}"
    if not os.path.exists(directory_targ + file.split('/')[-3] + os.sep+file.split('/')[-2]):
        os.makedirs(directory_targ+ file.split('/')[-3] + os.sep+file.split('/')[-2])
    download(file, directory_targ+file_name)
    txt_addresses.append(directory_targ+file_name)


2023-06-06 11:01:01,542-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:01,723-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:02,184-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:02,329-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:03,021-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:03,176-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:03,619-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:03,790-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:04,274-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:04,417-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:04,845-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:04,984-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:05,409-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:05,551-[INFO]- dropbox:474 -> Request to 

2023-06-06 11:01:42,288-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:42,441-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:42,862-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:43,009-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:43,675-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:43,816-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:44,198-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:44,348-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:44,855-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:44,995-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:45,525-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:45,668-[INFO]- dropbox:474 -> Request to files/download
2023-06-06 11:01:46,143-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-06 11:01:46,296-[INFO]- dropbox:474 -> Request to 

In [23]:
series_array = []
# print(txt_addresses)
for address in txt_addresses:
#     print(address)
    series = pd.read_csv(address, sep=": ", engine='python').dropna()
    series_array.append(series.T)
    
dataframe = pd.concat(series_array, axis=0, ignore_index=True)
dataframe["LocalPath"] = txt_addresses

,Computer,User,DateTime,StoragePath,Plate,Root,Strain,Treatment,CrossDate,Run,...,FrameSize,Binning,Gain,Gamma,Fiber Led,Fluorescence,Four Led Bar,X,Y,Z
0,LOUREED,Morrison-setup,"Friday, 12 May 2023, 10:49:11",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,047,None,A5,0Myristate,20220310,1,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-6.128 mm,37.600 mm,5.035 mm
1,LOUREED,Morrison-setup,"Friday, 12 May 2023, 10:50:07",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,047,None,A5,0Myristate,20220310,2,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-7.409 mm,38.195 mm,4.915 mm
2,LOUREED,Morrison-setup,"Friday, 12 May 2023, 10:51:01",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,047,None,A5,0Myristate,20220310,3,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-8.505 mm,38.737 mm,4.970 mm
3,LOUREED,Morrison-setup,"Friday, 12 May 2023, 10:54:11",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,047,None,A5,0Myristate,20220310,4,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-6.116 mm,37.432 mm,5.025 mm
4,LOUREED,Morrison-setup,"Friday, 12 May 2023, 10:55:41",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,047,None,A5,0Myristate,20220310,5,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-5.799 mm,39.153 mm,4.985 mm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,LOUREED,Morrison-setup,"Friday, 12 May 2023, 15:01:17",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,118,None,C2,0Myristate,20220324,8,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,5.419 mm,42.953 mm,4.615 mm
106,LOUREED,Morrison-setup,"Friday, 12 May 2023, 15:04:28",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,118,None,C2,0Myristate,20220324,9,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-8.155 mm,34.104 mm,5.050 mm
107,LOUREED,Morrison-setup,"Friday, 12 May 2023, 15:05:16",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,118,None,C2,0Myristate,20220324,10,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-8.380 mm,33.939 mm,5.080 mm
108,LOUREED,Morrison-setup,"Friday, 12 May 2023, 15:06:24",D:\AMOLF-SHIMIZU Dropbox\DATA\MYRISTATE\DA...,118,None,C2,0Myristate,20220324,11,...,2048x1500 8BPP,2x2,0.0,1.00,On,Off,Off,-7.731 mm,32.627 mm,4.990 mm


In [24]:
def month_to_num(x):
    months = {
        'jan': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
         'may':'05',
         'jun':'06',
         'jul':'07',
         'aug':'08',
         'sep':'09',
         'oct':'10',
         'nov':'11',
         'dec':'12'
        }
    a = x.strip()[:3].lower()
    try:
        ez = months[a]
        return ez
    except:
        raise ValueError('Not a month')

def reshape_video_info_df(video_info):
    clean_video_info = video_info.copy()
    clean_video_info["DateTime"] = [date.split(',')[-2] for date in clean_video_info["DateTime"]]
    clean_video_info["DateTime"] = [int(f"{date.split(' ')[-1]}{month_to_num(date.split(' ')[-2])}{date.split(' ')[-3]}") for date in clean_video_info["DateTime"]]
    clean_video_info["Plate"] = [str(entry) for entry in clean_video_info["Plate"]]
    clean_video_info["Time"] = [int(time.split(' ')[-2]) for time in clean_video_info["Time"]]
    clean_video_info["Magnification"] = [entry.split(" ")[-2][:-1] for entry in clean_video_info["Operation"]]
    clean_video_info["BF_or_F"] = [["F", "BF"][entry.split(" ")[-1] == 'Brightfield']for entry in clean_video_info["Operation"]]
#     for i, entry in enumerate(clean_video_info["X"]):
#         print(str(entry.lower()) == "auto")
#         print(entry.split(" "))
#         print(float(entry.split(" ")[14]))
#         print(clean_video_info["DateTime"][i])
#         print(clean_video_info["Run"][i])
    clean_video_info["ExposureTime"] = [entry.split(" ")[3] for entry in clean_video_info["ExposureTime"]]
    clean_video_info["FrameRate"] = [float(entry.split(" ")[-3]) for entry in clean_video_info["FrameRate"]]
    clean_video_info["Binning"] = [int(entry.split("x")[-1]) for entry in clean_video_info["Binning"]]
    clean_video_info["X"] = [float(entry.split(" ")[14]) for entry in clean_video_info["X"]]
    clean_video_info["Y"] = [float(entry.split(" ")[14]) for entry in clean_video_info["Y"]]
    clean_video_info["Z"] = [float(entry.split(" ")[14]) for entry in clean_video_info["Z"]]
    return clean_video_info


new_dataframe = reshape_video_info_df(dataframe)
new_dataframe["StoragePath"] = [address[:-13] + "Img/" for address in txt_drop]
new_dataframe.to_csv(f"{directory_targ}/PlateInfo.csv")

## Download section
